In [2]:
import warnings
import numpy as np
from six.moves import reload_module as reload
from sklearn.utils.extmath import fast_logdet
from sklearn.utils import check_random_state
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.covariance import empirical_covariance
from functools import partial

from network_inference.prox import prox_logdet, soft_thresholding_od
import network_inference.datasets; reload(network_inference.datasets)
from network_inference.datasets import is_pos_def, is_pos_semi_def
from network_inference.utils import _scalar_product, update_rho, convergence

# checks

In [18]:
H = np.random.rand(10,10)
H = H.T.dot(H)
is_pos_def(H)

True

In [19]:
KHK = K_true.dot(np.linalg.inv(H)).dot(K_true.T)
is_pos_semi_def(KHK)

True

In [5]:
np.random.seed(0)
random_state=0
A = make_sparse_spd_matrix(dim=15, alpha=0.7, random_state=random_state)

T_true = A[5:,5:]
K_true = A[10:,5:,]
H_true = A[0:5,0:5]

per_cov = K_true*0.3
T_obs = T_true - per_cov.T.dot(np.linalg.inv(H_true)).dot(per_cov)

samples = np.random.multivariate_normal(np.zeros(10), np.linalg.inv(T_obs), 50)

In [20]:
def objective_H(H, K=None, L=None, U=None,_rho=1, _mu=1):
    return _rho/2* np.linalg.norm(L + U+ K.T.dot(np.linalg.inv(H).dot(K)))**2 + _mu*np.linalg.norm(H, 1)


In [78]:
def _choose_lambda(lamda, H, K, L, U,  _rho, _mu, prox, grad, gamma, delta=1e-4, eps=0.9, max_iter=500):
    """Choose lambda for backtracking.

    References
    ----------
    Salzo S. (2017). https://doi.org/10.1137/16M1073741

    """
    lamda=1
    partial_f = partial(objective_H,K=K, L=L, U=U, _rho=_rho, _mu=_mu)
    fx = partial_f(H)

    y_minus_x = prox - H
    print(y_minus_x)
    tolerance = _scalar_product(y_minus_x, grad)
    print("Tolerance:", tolerance)
    
    tolerance += delta / gamma * _scalar_product(y_minus_x, y_minus_x)
    print("Tolerance:", tolerance)
    for i in range(max_iter):
        # line-search
        x1 = H + lamda * y_minus_x

        loss_diff = partial_f(x1) - fx
        print("Loss diff:", loss_diff)
        if loss_diff <= lamda * tolerance and is_pos_def(x1):
              break
        lamda *= eps
    else:
        print("nope lambda")
    return lamda, i + 1

In [88]:
def _choose_gamma(H, _mu, grad, eps=0.9, max_iter=500):
    """Choose gamma for backtracking.

    References
    ----------
    Salzo S. (2017). https://doi.org/10.1137/16M1073741

    """
    gamma=1
    for i in range(max_iter):
        prox = soft_thresholding_od(H - gamma * grad, _mu * gamma)
        if is_pos_def(prox):
            print("cacca")
            break
        gamma *= eps
    else:
        print("nope gamma")
    return gamma, prox

In [89]:
def _upgrade_H(L, K, U, _rho, _mu, verbose=0):
    H = np.random.rand(K.shape[0], K.shape[0])
    H = (H + H.T)/2
    _lambda = 1
    gamma = 1
    obj = 1e+10
    print(H.shape)
    for iter_ in range(1000):
        #print(H)
        H_old = H
        Hinv = np.linalg.inv(H)
        gradient = -_rho* K.dot(L + U - np.linalg.multi_dot((K.T, Hinv, K))).dot(K.T).dot(Hinv).dot(Hinv)
        #print(gradient)
        gamma, _ = _choose_gamma(H, _mu, gradient)
        print(gamma)
        #gamma=1
        Y = soft_thresholding_od(H - gamma*gradient, gamma*_mu)
        #print(Y)
        _lambda,_ = _choose_lambda(_lambda, H,K,L,U,_rho, _mu, Y, gradient, 1, max_iter=1000, delta=1e-2)
        print(_lambda)
        H = H + _lambda*(Y - H)
        obj_old = obj
        obj = objective_H(H, K, L, U,_rho=_rho, _mu=_mu)
        obj_diff = obj_old - obj
        iter_diff =np.linalg.norm(H - H_old) 
        if verbose:
            print("Iter: %d, obj: %.5f, iter_diff: %.5f, obj_diff:%.10f"%(iter_, obj, iter_diff, obj_diff))
        if(obj_diff<1e-2 and iter_ > 50): 
            break
    else:
        print("Did not converge")
    return H

In [90]:
K = per_cov
H = np.random.rand(5,5)
L = K.T.dot(np.linalg.pinv(H)).dot(K)
U = np.zeros((10,10))
H_found = _upgrade_H(L, K, U, 1,0.5, 1)
H - H_found
#print(H_found)

(5, 5)
nope gamma
1.3220708194808237e-23
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Tolerance: 0.0
Tolerance: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.

Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff:

Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff:

Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff:

Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
nope lambda
1.7478712517226966e-46
Iter: 2, obj: 13.62454, iter_diff: 0.00000, obj_diff:0.0000000000
nope gamma
1.3220708194808237e-23
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Tolerance: 0.0
Tolerance: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff

...

Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff:

Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff:

Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff:

Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff: 0.0
Loss diff:

array([[ 0.25637346, -0.38692862,  0.00356688,  0.69283561,  0.19279498],
       [-0.28629216, -0.63308514,  0.44450104, -0.26439763, -0.36064664],
       [-0.30673447, -0.03225758,  0.88094714,  0.40718508, -0.02500858],
       [ 0.05195736, -0.14217585, -0.57124995, -0.47443276,  0.25275196],
       [-0.6103583 , -0.55305126,  0.09951171,  0.04814319, -0.54181812]])

In [66]:
def objective(emp_cov, K, R, T, H, mu, eta, rho):
    res = fast_logdet(R)
    res += np.sum(R*emp_cov)
    res += rho/2 * np.linalg.norm(R - T + U + K.T.dot(np.linalg.inv(H)).dot(K))**2 
    res += mu*np.linalg.norm(H,1)
    res += eta*np.linalg.norm(T,1)
    return res

In [82]:
def fixed_interlinks_graphical_lasso(X, K, mu=0.01, eta=0.01, rho=1., 
        tol=1e-3, rtol=1e-5, max_iter=100, verbose=False, return_n_iter=True,
        return_history=False, compute_objective=False, compute_emp_cov=False,
        random_state=None):
    
    random_state = check_random_state(random_state)
    if compute_emp_cov:
        n = X.shape[0] 
        emp_cov = empirical_covariance(X, assume_centered=False)
    else:
        emp_cov = X

    H = np.random.rand(K.shape[0], K.shape[0])
    H = H.T.dot(H)/2
    T = emp_cov.copy()
    T = (T + T.T)/2
    R = T - np.linalg.multi_dot((K.T, np.linalg.pinv(H), K))
    U = np.zeros((K.shape[1], K.shape[1]))
    
    checks = []
    for iteration_ in range(max_iter):
        R_old = R.copy()
        
        # R update
        M = T - U - K.T.dot(np.linalg.pinv(H)).dot(K)
        M = (M + M.T)/2
        R = prox_logdet(emp_cov - rho*M, 1/rho)
        assert is_pos_def(R), "iter %d"%iteration_
        #print("----------------------R---------------------\n", R)
        # T update
        M = - R - U - K.T.dot(np.linalg.pinv(H)).dot(K)
        M = (T + T.T)/2
        T = soft_thresholding_od(M, eta/rho)
        assert is_pos_def(T), "teta iter %d"%iteration_
       
        #print("----------------------T---------------------\n",T)
        # H update
        H = _upgrade_H(R, T, K, U, rho, mu)
        assert(is_pos_def(H))
        #print("----------------------H---------------------\n",H)
        # U update
        KHK = np.linalg.multi_dot((K.T, np.linalg.pinv(H), K))
        assert is_pos_semi_def(KHK)
        U += R - T + KHK

        # diagnostics, reporting, termination checks
        
        obj = objective(emp_cov, K, R, T, H, mu, eta, rho) \
            if compute_objective else np.nan
        rnorm = np.linalg.norm(R - T + KHK)
        snorm = rho *np.linalg.norm(R - R_old)
        check = convergence(
            obj=obj, rnorm=rnorm, snorm=snorm,
            e_pri=(np.sqrt(R.size) * tol + rtol *
                   max(np.sqrt(np.linalg.norm(R)**2 + np.linalg.norm(U)**2),
                       np.linalg.norm(T - KHK))),
            e_dual=(np.sqrt(R.size) * tol + rtol * rho *
                    np.linalg.norm(U))
        )

        if verbose:
            print("obj: %.4f, rnorm: %.4f, snorm: %.4f,"
                  "eps_pri: %.4f, eps_dual: %.4f" % check)

        checks.append(check)
        if check.rnorm <= check.e_pri and check.snorm <= check.e_dual:
            break
        rho_new = update_rho(rho, rnorm, snorm, iteration=iteration_)
        # scaled dual variables should be also rescaled
        U *= rho / rho_new
        rho = rho_new
    else:
        warnings.warn("Objective did not converge.")

    return_list = [R, T, H, emp_cov]
    if return_n_iter:
        return_list.append(iteration_)
    if return_history:
        return_list.append(checks)
    return return_list


In [83]:
res = fixed_interlinks_graphical_lasso(samples, per_cov, mu=1, eta=1, rho=1., 
        verbose=1, compute_objective=1, compute_emp_cov=1,
        random_state=0)

AssertionError: 